In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import requests
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from datetime import datetime
import re
import time
import sys
import tqdm
import selenium.common.exceptions as sce

from getpass import getpass

In [2]:
input_data = pd.read_csv('./naver_map_seoul_salon_output_data_2020-04-29_final.csv', index_col=0)

In [3]:
input_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13398 entries, 0 to 13397
Data columns (total 8 columns):
store_id             13398 non-null object
store_id_only_num    13398 non-null int64
store_name           13398 non-null object
tel                  9515 non-null object
addrRoad             13398 non-null object
category             13398 non-null object
collect_day          13398 non-null object
disctrict            13398 non-null object
dtypes: int64(1), object(7)
memory usage: 942.0+ KB


In [4]:
input_data.head()

,store_id,store_id_only_num,store_name,tel,addrRoad,category,collect_day,disctrict
0,s18570093,18570093,조희미용실,02-555-0157,서울특별시 강남구 남부순환로 2917 1층 111-가,"생활,편의 > 미용실",2020-04-19,강남구
1,s10983326,10983326,정정원 헤어 룩,02-545-3809,서울특별시 강남구 선릉로116길 24,"생활,편의 > 미용실",2020-04-19,강남구
2,s18431392,18431392,헤어인러브,02-546-0366,서울특별시 강남구 선릉로148길 39 2층,"생활,편의 > 미용실",2020-04-19,강남구
3,s34786519,34786519,앤즈헤어,02-553-9799,서울특별시 강남구 언주로 201 sk리더스뷰 2층 앤즈헤어,"생활,편의 > 미용실",2020-04-19,강남구
4,s697084146,697084146,아나스타일,02-501-0457,서울특별시 강남구 언주로97길 25,"생활,편의 > 미용실",2020-04-19,강남구


In [5]:
nan_data = pd.read_csv('./naver_blog_count_nan_data_2020-04-30.csv', index_col=0)

In [6]:
nan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 13327 to 13393
Data columns (total 7 columns):
store_id             67 non-null object
store_id_only_num    67 non-null int64
store_name           67 non-null object
booking              0 non-null float64
receipt              0 non-null float64
blog                 0 non-null float64
total                67 non-null float64
dtypes: float64(4), int64(1), object(2)
memory usage: 4.2+ KB


In [7]:
nan_data.head()

,store_id,store_id_only_num,store_name,booking,receipt,blog,total
13327,s325908825,325908825,데꼬헤살롱,NaN,NaN,NaN,0.0
13328,s11868905,11868905,렉시온헤어,NaN,NaN,NaN,0.0
13329,s1824822666,1824822666,멋지다 당신샵,NaN,NaN,NaN,0.0
13330,s18605329,18605329,헤어마을,NaN,NaN,NaN,0.0
13331,s32613863,32613863,리안헤어 신대방역점,NaN,NaN,NaN,0.0


In [8]:
search_data = pd.merge(nan_data, input_data, left_on='store_id', right_on='store_id')
search_data.head()

,store_id,store_id_only_num_x,store_name_x,booking,receipt,blog,total,store_id_only_num_y,store_name_y,tel,addrRoad,category,collect_day,disctrict
0,s325908825,325908825,데꼬헤살롱,NaN,NaN,NaN,0.0,325908825,데꼬헤살롱,02-6489-1010,서울특별시 서초구 효령로77길 34 강남아크로텔 지하1층 113호114호 데꼬헤살롱,"생활,편의 > 미용실",2020-04-19,서초구
1,s11868905,11868905,렉시온헤어,NaN,NaN,NaN,0.0,11868905,렉시온헤어,02-956-1105,서울특별시 도봉구 도봉로152길 16,"생활,편의 > 미용실",2020-04-19,도봉구
2,s1824822666,1824822666,멋지다 당신샵,NaN,NaN,NaN,0.0,1824822666,멋지다 당신샵,NaN,서울특별시 광진구 능동로40길 19,"생활,편의 > 미용실",2020-04-19,광진구
3,s18605329,18605329,헤어마을,NaN,NaN,NaN,0.0,18605329,헤어마을,02-982-6675,서울특별시 강북구 인수봉로12길 63,"생활,편의 > 미용실",2020-04-19,강북구
4,s32613863,32613863,리안헤어 신대방역점,NaN,NaN,NaN,0.0,32613863,리안헤어 신대방역점,02-861-3336,서울특별시 관악구 난곡로 367 경신빌딩,"생활,편의 > 미용실",2020-04-19,관악구


In [9]:
def start(start_store_name: str):
    options = webdriver.ChromeOptions()

    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')
    options.add_argument("Accept=text/html,application/xhtml+xml,application/xml;q=0.9,imgwebp,*/*;q=0.8")
    
    driver = webdriver.Chrome('c:/chromedriver', options=options)
    driver.get('https://m.naver.com/')
    
    time.sleep(1)
    input_box = driver.find_element_by_id('MM_SEARCH_FAKE')
    
    time.sleep(0.5)
    input_box.click()
    driver.find_element(By.ID, 'query').send_keys(start_store_name)
    
    driver.find_element_by_class_name('ico_sch_submit').submit()
    
    return driver

In [10]:
def get_addrs(driver):
    elems = driver.find_elements(By.CLASS_NAME, '_2TY1G ')
    addrs = []
    try:
        for elem in elems:
            arrow = elem.find_element(By.CLASS_NAME, '_1Ck7n')
            arrow.click()
            addrs.append(driver.find_element(By.CLASS_NAME, 'afoHe').text[6:-2])
            arrow.click()
    except sce.NoSuchElementException:
        addrs = []
    finally:
        return addrs

In [11]:
def get_index_addr_match(addrs: list, addrRoad: str):
    for i, addr in enumerate(addrs):
        if re.sub(' ', '', addr) in re.sub(' ', '', addrRoad):
            return i
        else:
            continue
    return None

In [12]:
def click_more_btn(driver):
    try:
        driver.find_element(By.CLASS_NAME, '_3iTUo').click()
        time.sleep(0.2)
        click_more_btn(driver)
    except sce.NoSuchElementException:
        return

In [13]:
# id 값 가져오기
def get_store_id(driver):
    url = driver.current_url
    id = list(filter(lambda x: x.isdigit(), url.split('/')))[0]
    return id

In [14]:
def search_keyword(driver, keyword):
    input_box = driver.find_element(By.ID, 'nx_query')
    input_box.click()
    input_box.clear()
    input_box.send_keys(keyword)
    
    driver.find_element(By.CLASS_NAME, 'btn_search').submit()

In [15]:
def get_place_info(driver, row_index):

    # 네이버 플레이스 id 가져오기
    place_store_id = get_store_id(driver)
    # 네이버 플레이스 업소명 가져오기
    place_store_name = driver.find_element(By.CLASS_NAME, '_3XamX').text
    print('Get place id and name')    

    try:
        # 리뷰탭 들어가기 (기본 예약자 리뷰)
        tabs = driver.find_elements(By.CLASS_NAME, 'H04yZ')
        review_tab = list(filter(lambda x: x.text == '리뷰' , tabs))
        if review_tab:
            review_tab[0].click()
        else:
            count_booking = np.nan
            count_receipt = np.nan
            count_blog = np.nan
            place_info = [place_store_id, place_store_name, count_booking, count_receipt, count_blog]
            print('No review tab (tabs exist)')
            return place_info
    except sce.NoSuchElementException:
        count_booking = np.nan
        count_receipt = np.nan
        count_blog = np.nan
        place_info = [place_store_id, place_store_name, count_booking, count_receipt, count_blog]
        print('No review tab (tabs NOT exist)')
        return place_info
    
    try:
        # 예약자 리뷰탭 들어가기
        xpath_review = """//*[@id="app-root"]/div/div[2]/div[4]/div[1]/div/a[1]"""
        driver.find_element(By.XPATH, xpath_review).click()
        time.sleep(1.5)
        click_more_btn(driver)
        count_booking = len(driver.find_elements(By.CLASS_NAME, 'JIiPr'))
        print('Get booking review count')
    except sce.NoSuchElementException:
        count_booking = np.nan
        print('No booking review')
    
    
    try:
        # 영수증 리뷰 탭 들어가기
        xpath_receipt = """//*[@id="app-root"]/div/div[2]/div[4]/div[1]/div/a[2]"""
        driver.find_element(By.XPATH, xpath_receipt).click()
        time.sleep(1.5)
        click_more_btn(driver)
        count_receipt = len(driver.find_elements(By.CLASS_NAME, '_1-41P'))
        print('Get receipt review count')
    except sce.NoSuchElementException:
        count_receipt = np.nan
        print('No receipt review')    
    
    try:
        # 블로그 리뷰 탭 들어가기
        xpath_blog = """//*[@id="app-root"]/div/div[2]/div[4]/div[1]/div/a[3]"""
        driver.find_element(By.XPATH, xpath_blog).click()
        time.sleep(1.5)
        click_more_btn(driver)
        count_blog = len(driver.find_elements(By.CLASS_NAME, '_3xK_0'))
        print('Get blog review count')
    except sce.NoSuchElementException:
        count_blog = np.nan
        print('No blog review')
        
    # list 생성
    place_info = [place_store_id, place_store_name, count_booking, count_receipt, count_blog]
    
    print(f'[row {row_index}] Get Naver Place Info SUCCESS')
    
    return place_info

In [16]:
def check_place_match(driver, addrs: list, addrRoad: str):

    place_elems_one = driver.find_elements(By.ID, 'place-main-section-root')
    place_elems_two = driver.find_elements(By.CLASS_NAME, 'place_section_header no_line type_nx')

    if place_elems_one:
        driver.find_element(By.CLASS_NAME, '_3XamX').click()
        status = 1
    elif place_elems_two:
        driver.find_element(By.CLASS_NAME, '_1hEhO').click()
        status = 1
    elif addrs: 
        index = get_index_addr_match(addrs, addrRoad)
        if index:
            driver.find_elements(By.CLASS_NAME, '_1hEhO')[index].click()
            status = 1
        else:
            print('지도 정보에서 매치된 미용실 없음')
            status = 0
    else:
        print('해당 플레이스 정보 없음')
        status = 0
    
    return status

In [17]:
###############################################################

In [20]:
tmp = input_data[input_data.store_name == '준오헤어 노원사거리점']

In [407]:
driver.quit()

In [23]:
# def main(search_data: DataFrame):
place_infos = []
place_exist = 0
for row_index in tqdm.tnrange(len(tmp)):
    store_id = tmp.iloc[row_index].store_id
    store_name = tmp.iloc[row_index].store_name
    addrRoad = tmp.iloc[row_index].addrRoad
    
    print(f'[row {row_index}] {store_id} {store_name}')
    
    # 첫번째 검색
    if row_index == 0:
        driver = start(store_name)
    # 두번째 이후 검색
    else:
        if place_exist:
            # 검색창으로 돌아가기
#             xpath_goback_search = """//*[@id="app-root"]/div/div[2]/footer/a"""
#             driver.find_element(By.CLASS_NAME, '_1NKY0').click()
            w = WebDriverWait(driver, 15)
            elem = w.until(EC.element_to_be_clickable((By.CLASS_NAME, '_1NKY0')))
            try:
                #elem.click()
                #time.sleep(0.5)
                #elem.click()
                elem.send_keys(Keys.ENTER)

            except sce.StaleElementReferenceException:            
                pass
            print('통합검색 결과보기 클릭!')
            time.sleep(1)
        else:
            pass

        # 검색
        search_keyword(driver, store_name)

    addrs = get_addrs(driver)
    status = check_place_match(driver, addrs, addrRoad)

    if status:
        place_exist = 1
        place_info = get_place_info(driver, row_index)
    else:
        place_exist = 0
        place_info = [np.nan] * 5
    
    print('place_exist :', place_exist)
    place_info = [store_id, store_name] + place_info
    print(place_info, '\n')
    place_infos.append(place_info)

naver_search_result = DataFrame(place_infos, columns=['store_id', 'store_name', 'place_store_id', 'place_store_name', 'booking', 'receipt', 'blog'])
naver_search_result

[row 0] s1518997092 준오헤어 노원사거리점
Get place id and name
Get booking review count
No receipt review
No blog review
[row 0] Get Naver Place Info SUCCESS
place_exist : 1
['s1518997092', '준오헤어 노원사거리점', '1518997092', '준오헤어 노원사거리점', 0, nan, nan] 




,store_id,store_name,place_store_id,place_store_name,booking,receipt,blog
0,s1518997092,준오헤어 노원사거리점,1518997092,준오헤어 노원사거리점,0,NaN,NaN


In [ ]:
place_infos

In [ ]:
naver_search_result

In [409]:
driver.execute_script("return window.pageYOffset;")

0

In [336]:
driver.quit()

In [337]:
# def main(search_data: DataFrame):
place_infos = []
place_exist = 0

row_index = 0

store_id = search_data[1:].iloc[row_index].store_id
store_name = search_data[1:].iloc[row_index].store_name_x
addrRoad = search_data[1:].iloc[row_index].addrRoad

print(f'[row {row_index}] {store_id} {store_name}')

# 첫번째 검색
if row_index == 0:
    driver = start(store_name)
# 두번째 이후 검색
else:
    if place_exist:
        # 검색창으로 돌아가기
#             xpath_goback_search = """//*[@id="app-root"]/div/div[2]/footer/a"""
#             driver.find_element(By.CLASS_NAME, '_1NKY0').click()
        w = WebDriverWait(driver, 15)
        elem = w.until(EC.element_to_be_clickable((By.CLASS_NAME, '_1NKY0')))
        elem.click()
        print('통합검색 결과보기 클릭!')
        time.sleep(1)
    else:
        pass

    # 검색
    search_keyword(driver, store_name)

addrs = get_addrs(driver)
status = check_place_match(driver, addrs, addrRoad)

if status:
    place_exist = 1
    place_info = get_place_info(driver, row_index)
else:
    place_exist = 0
    place_info = [np.nan] * 5

print('place_exist :', place_exist)
place_info = [store_id, store_name] + place_info
print(place_info, '\n')
place_infos.append(place_info)


[row 0] s11868905 렉시온헤어
해당 플레이스 정보 없음
place_exist : 0
['s11868905', '렉시온헤어', nan, nan, nan, nan, nan] 



In [338]:
row_index = 1

store_id = search_data[1:].iloc[row_index].store_id
store_name = search_data[1:].iloc[row_index].store_name_x
addrRoad = search_data[1:].iloc[row_index].addrRoad

print(f'[row {row_index}] {store_id} {store_name}')

# 첫번째 검색
if row_index == 0:
    driver = start(store_name)
# 두번째 이후 검색
else:
    if place_exist:
        # 검색창으로 돌아가기
#             xpath_goback_search = """//*[@id="app-root"]/div/div[2]/footer/a"""
#             driver.find_element(By.CLASS_NAME, '_1NKY0').click()
        w = WebDriverWait(driver, 15)
        elem = w.until(EC.element_to_be_clickable((By.CLASS_NAME, '_1NKY0')))
        elem.click()
        print('통합검색 결과보기 클릭!')
        time.sleep(1)
    else:
        pass

    # 검색
    search_keyword(driver, store_name)

addrs = get_addrs(driver)
status = check_place_match(driver, addrs, addrRoad)

if status:
    place_exist = 1
    place_info = get_place_info(driver, row_index)
else:
    place_exist = 0
    place_info = [np.nan] * 5

print('place_exist :', place_exist)
place_info = [store_id, store_name] + place_info
print(place_info, '\n')
place_infos.append(place_info)


[row 1] s1824822666 멋지다 당신샵
Get place id and name
No review tab (tabs exist)
place_exist : 1
['s1824822666', '멋지다 당신샵', '1200071573', '멋지다당신', nan, nan, nan] 



In [334]:
row_index = 2

store_id = search_data[1:].iloc[row_index].store_id
store_name = search_data[1:].iloc[row_index].store_name_x
addrRoad = search_data[1:].iloc[row_index].addrRoad

print(f'[row {row_index}] {store_id} {store_name}')

[row 2] s18605329 헤어마을


In [357]:
w = WebDriverWait(driver, 15)
elem = w.until(EC.element_to_be_clickable((By.CLASS_NAME, '_1NKY0')))
elem.click()
# print('통합검색 결과보기 클릭!')
# time.sleep(1)

In [358]:
elem.click()

In [ ]:
# 검색
search_keyword(driver, store_name)

addrs = get_addrs(driver)
status = check_place_match(driver, addrs, addrRoad)

if status:
    place_exist = 1
    place_info = get_place_info(driver, row_index)
else:
    place_exist = 0
    place_info = [np.nan] * 5

print('place_exist :', place_exist)
place_info = [store_id, store_name] + place_info
print(place_info, '\n')
place_infos.append(place_info)

In [36]:
driver.find_element(By.ID, 'query').send_keys(start_store_name)

In [ ]:
driver.find_element_by_class_name('ico_sch_submit').submit()

In [ ]:
main(search_data)

In [ ]:
xpath_booking = """//*[@id="app-root"]/div/div[2]/div[4]/div/div[9]/div[2]/a"""

In [263]:
xpath_review = """//*[@id="app-root"]/div/div[2]/div[3]/div/div/a[6]"""

In [264]:
driver.find_element(By.XPATH, xpath_review).click()

In [265]:
xpath_more_btn = """//*[@id="app-root"]/div/div[2]/div[4]/div[2]/div[4]/div[2]/a"""

In [297]:
xpath_reciept = """//*[@id="app-root"]/div/div[2]/div[4]/div[1]/div/a[2]"""
driver.find_element(By.XPATH, xpath_reciept).click()

In [298]:
xpath_blog = """//*[@id="app-root"]/div/div[2]/div[4]/div[1]/div/a[3]"""
driver.find_element(By.XPATH, xpath_blog).click()

In [304]:
click_more_btn(driver)

'325908825'

In [ ]:
# 지도정보
'_2g5fr '

In [251]:
name = driver.find_elements(By.CLASS_NAME, '_1hEhO')
len(name)

5

In [88]:
next_input_box = driver.find_element_by_id('nx_query')

In [199]:
start_store_name = search_data.iloc[0].store_name_x
start_store_name

'헤어마을'

In [95]:
#input_box.clear()
#input_box.send_keys(tmp_store_name)

next_input_box.clear()
next_input_box.send_keys(tmp_store_name)

In [97]:
# 검색버튼 
#driver.find_element_by_class_name('ico_sch_submit').submit()

# 다음 검색 버튼
driver.find_element_by_class_name('btn_search').submit()

In [185]:
# # 업소이름
# title = driver.find_elements(By.CLASS_NAME, 'es3Ot')
# len(title)

5

In [84]:
# 검색결과화면 +더보기
def click_more_btn_until_end(driver):
    try:
        driver.find_element_by_class_name('_1kcg0').click()
        time.sleep(1)
        click_more_btn_until_end(driver)
    except sce.NoSuchElementException:
        return

no result


In [68]:
# id 값 가져오기
id = list(filter(lambda x: x.isdigit(), url.split('/')))[0]
id

'325908825'

In [41]:
# 블로그 리뷰 더보기
driver.find_element_by_class_name('H04yZ').click()

In [ ]:
# 예약자 리뷰 탭 : _3ZOC7
# 영수증 리뷰 탭 : _3ZOC7
# 블로그 리뷰 탭 : _3ZOC7

In [42]:
reivew_tabs = driver.find_elements_by_class_name('_3ZOC7')

In [43]:
# 블로그 리뷰로 들어가기
reivew_tabs[2].click()

In [63]:
# 더보기 버튼
try:
    driver.find_element_by_class_name('_3iTUo').click()
except sce.NoSuchElementException:
    pass

In [61]:
reviews = driver.find_elements_by_class_name('_3xK_0')

In [62]:
len(reviews)

100

In [66]:
url = driver.current_url
url

'https://m.place.naver.com/hairshop/325908825/review/ugc'

In [67]:
list(filter(lambda x: x.isdigit(), url.split('/')))[0]

'325908825'

In [13]:
# driver.quit()